In [1]:
import torch
import torch.distributed as dist
from torch import nn
from torch.cuda.amp import autocast
from torch.utils.data import DataLoader, Dataset, DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import CLIPTokenizer, CLIPTextModel
from diffusers import UNet2DConditionModel, AutoencoderKL, StableDiffusionPipeline

# Initialize Distributed Training
def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

/data/maryam.sana/anaconda3/envs/compressai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to('cuda')

Loading pipeline components...: 100%|████████████████████████████| 7/7 [00:01<00:00,  6.45it/s]


In [3]:
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")          
pipe.vae = vae

In [5]:
with open('results/caption.txt', 'r') as file:
    # Read the contents of the file
    caption = file.read()

In [4]:
image_orig = Image.open('data/Jockey/img00001.png')
# image = pipe(
# 	caption, height=image_orig.size[1], width=image_orig.size[0]
# ).images[0]
# image

In [14]:
from utils import *

print('MSE:', mse_loss(image_orig, image))

psnr = calculate_psnr(image_orig, image)
print('PSNR:', psnr)

reconstructed_bpp = calculate_bpp('results/caption.txt', image.size)
print("Bits per pixel of reconstructed image:", reconstructed_bpp)

MSE: 104.71739776234568
PSNR: 27.930615194517937
Bits per pixel of reconstructed image: 0.0016126543209876542


In [4]:
transform = transforms.ToTensor()
tensor1 = transform(image_orig)
tensor2 = transform(image)
F.mse_loss(tensor1, tensor2)

NameError: name 'image_orig' is not defined

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32",torch_dtype=torch.float16)  
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32",torch_dtype=torch.float16).to(device) 
unet = pipe.unet.half()
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")  
vae= vae.half()

cuda


In [26]:
class TextImageDataset(Dataset):
    def __init__(self, texts, image_paths, tokenizer, transform=None):
        self.texts = texts
        self.image_paths = image_paths
        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        inputs = self.tokenizer(text, return_tensors="pt")
        return inputs.input_ids.squeeze(0), image

In [27]:
optimizer = torch.optim.Adam(list(unet.parameters()) + list(vae.parameters()), lr=1e-5)

In [28]:
transform_to_tensor = transforms.ToTensor()

# Custom collate function to pad text inputs and convert images to tensors
def collate_batch(batch):
    text_inputs, images = zip(*batch)
    text_inputs_padded = pad_sequence(text_inputs, batch_first=True, padding_value=0)
    images = torch.stack([transform_to_tensor(img) for img in images], dim=0)
    return text_inputs_padded, images

# Assuming 'dataset' is already defined
dataset = TextImageDataset(texts=["A rider riding a horse on a track", "A rider moving forward on the track with horse"], image_paths=["data/Jockey/img00100.png", "data/Jockey/img00001.png"], tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_batch)

In [31]:
from torch.cuda.amp import autocast
def train(models, dataloader, epochs):
    optimizer = torch.optim.Adam([p for model in models for p in model.parameters()], lr=1e-4)  # Define optimizer

    for epoch in range(epochs):
        for text_inputs, images in dataloader:
            with autocast():
                text_inputs = text_inputs.to(device)
                images = images.to(device)
                
                # Encode text inputs and images
                text_features = models[0](text_inputs).last_hidden_state
                latents = models[2].encode(images)[0]
                
                # Generate images with UNet
                generated_images = models[1](text_features,encoder_hidden_states=latents).sample()  # Adjust order if needed
                
                # Calculate loss and update model
                loss = torch.nn.functional.mse_loss(generated_images, images)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            print(f"Epoch {epoch}, Loss: {loss.item()}")

In [32]:
train([text_encoder, unet, vae], dataloader, epochs=2)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.98 GiB. GPU 0 has a total capacity of 47.54 GiB of which 1.18 GiB is free. Including non-PyTorch memory, this process has 46.35 GiB memory in use. Of the allocated memory 45.66 GiB is allocated by PyTorch, and 368.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)